In [ ]:
import requests as rq
from bs4 import BeautifulSoup as bfs
from IPython.display import HTML
import pandas as pd
import numpy as np
import time
import random

# Get all acode (runner unique code) in a run event

In [ ]:
HTML('<iframe src=https://services.datasport.com/2015/lauf/transviamala/alfaw.htm width=1000 height=350></iframe>')

In [ ]:
def get_all_acode(url):
    page = rq.get(url)
    page_soup = bfs(page.text, 'html5lib')

    runners = page_soup.body.findAll('span', attrs={'class': 'myds'})
    
    return [runner['acode'] for runner in runners]

def get_all_acode_from_run_event(url):
    data = []
    page = rq.get(url)
    soup = bfs(page.text, 'html5lib')
    table_links = soup.select('font > a[href*=ALF]')
    all_acode = []
    for idx, link in enumerate(table_links):
        full_link = url + link['href']
        
        print(str(idx+1) + '/' + str(len(table_links)) + ' - Processing ' + full_link)

        # Remove duplicate entries and merges acodes
        #all_acode = all_acode + list(set(get_all_acode(full_link)) - set(all_acode))
        all_acode += get_all_acode(full_link)
        
        time.sleep(random.uniform(0.5, 2))

    return all_acode

In [ ]:
acode_list = get_all_acode_from_run_event('https://services.datasport.com/2015/lauf/transviamala/')

# Get run events

In [ ]:
HTML('<iframe src=https://www.datasport.com/fr/calendrier/ width=1000 height=350></iframe>')

## Get all params

In [ ]:
def get_all_params_calendar():
    calendar_page = rq.post('https://www.datasport.com/fr/calendrier/', data = calendar_params)
    calendar_soup = bfs(calendar_page.text, 'html5lib')

    selector_table = calendar_soup.find('table', attrs={'id': 'ds-calendar-header'})

    available_params = {}
    for selector in selector_table.findAll('select'):
        available_params[selector['name']] = {option.text: option['value'] for option in selector.findAll('option')}

    return available_params

In [ ]:
calendar_available_params = get_all_params_calendar()
calendar_available_params

## Get all run events url in a calendar

In [ ]:
def get_run_events_url(year=2009, month=1, country='CCH', sport='Running'):

    calendar_params = {
        'dr': '',
        'lastQuery': 'D147BC896417D2D2B96FA1AADD893731',
        'eventsearch': '',
        'eventservice': 'all',
        'start': 1,
        'etyp': sport,
        'eventlocation': country,
        'eventmonth': month,
        'eventyear': year,
    }

    calendar_page = rq.post('https://www.datasport.com/fr/calendrier/', data = calendar_params)
    calendar_soup = bfs(calendar_page.text, 'html5lib')

    table = calendar_soup.find('table', attrs={'id': 'ds-calendar-body'})

    all_event_url = []
    for row in table.findAll('tr'):
        columns = row.findAll('td')
        if len(columns) >= 4:
            url = columns[4].find('a')
            if url:
                all_event_url.append(url['href'])

    return all_event_url

In [ ]:
get_run_events_url()

## Get all run event urls

In [ ]:
def get_all_run_events():
    run_events = []
    for year_text, year_value in calendar_available_params['eventyear'].items():
        if year_value != 'all':
            for month_text, month_value in calendar_available_params['eventmonth'].items():
                if month_value != 'all':
                    print('Year ' + year_value + ' / Month ' + month_value)
                    run_events += get_run_events_url(year_value, month_value)
                    time.sleep(random.uniform(0.5, 2))
    return run_events

In [ ]:
run_events = get_all_run_events()
run_events